In [ ]:
!pip install konlpy

import requests
import csv
import pandas as pd
from konlpy.tag import Okt
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 45.4 MB/s eta 0:00:00


# 뉴스/블로그 크롤링

In [ ]:
# Naver API 인증 정보
client_id = '3d97MELhJ7QMyCT5GNoL'
client_secret = 'DmEHKmu_Ar'

keywords = ["중장년 취미 생활", "퇴직 후 재테크", "중장년 요리 클래스", "퇴직 후 운동 프로그램", "중장년 예술 활동", "퇴직 후 동호회",
            "퇴직 후 자원봉사", "퇴직 후 가족 시간", "중장년 사회 활동", "퇴직 후 자연 탐방", "퇴직 후 자산 관리", "중장년 명상과 요가",
            "퇴직 후 음악 활동", "중장년 온라인 강의", "중장년 퇴직 후 일자리", "퇴직자 재취업", "퇴직 후 창업", "중장년 일자리 추천", "퇴직자 일자리",
            "중장년 재취업 지원", "퇴직 후 교육 프로그램", "중장년 프리랜서", "퇴직 후 자영업", "중장년 아르바이트", "퇴직 후 재능 기부", "중장년 직업 훈련",
            "퇴직자 일자리 박람회", "중장년 일자리 매칭", "퇴직 후 온라인 강의", "중장년 일자리 플랫폼", "중장년 취업 성공 사례", "퇴직 후 재택 근무",
            "중장년 일자리 정보 사이트", "중장년 인기 직업", "퇴직 후 인기 직업", "퇴직자 인기 직업", "퇴직 후 인기 관심사", "퇴직자 인기 관심사", "퇴직자 취미",
            "은퇴 취미", "은퇴 관심사", "중장년 취미", "중장년 관심사", "중장년 직업 순위", "중장년 관심사 순위",
            "은퇴 직업 순위", "은퇴 관심사 순위", "퇴직 직업 순위", "퇴직 관심사 순위", "노년 직업 순위", "노년 관심사 순위"]

# 헤더 정보
headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

filename = f"뉴스_블로그_크롤링.csv"
with open(filename, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['검색어', '출처', '제목', '설명', '링크'])

    for query in keywords:  # keywords 리스트를 사용하여 검색어 설정
        for start in range(1, 1001, 100):  # 시작 위치를 1부터 1000까지 100씩 증가
            news_url = f"https://openapi.naver.com/v1/search/news.json?query={query}&start={start}&display=100"  # display를 100으로 설정
            blog_url = f"https://openapi.naver.com/v1/search/blog.json?query={query}&start={start}&display=100"

            # 뉴스 API 요청
            response_news = requests.get(news_url, headers=headers)
            # 블로그 API 요청
            response_blog = requests.get(blog_url, headers=headers)

            # 뉴스 응답 확인 및 데이터 처리
            if response_news.status_code == 200:
                data_news = response_news.json()
                items_news = data_news['items']

                if not items_news:  # 검색 결과가 더 이상 없으면 반복 중단
                    break

                for item in items_news:
                    title = item['title'].replace('<b>', '').replace('</b>', '')
                    description = item['description'].replace('<b>', '').replace('</b>', '')
                    link = item['link']
                    writer.writerow([query, '뉴스', title, description, link])

            else:
                print(f"뉴스 에러 코드: {response_news.status_code}")
                break  # 에러 발생 시 반복 중단

            # 블로그 응답 확인 및 데이터 처리
            if response_blog.status_code == 200:
                data_blog = response_blog.json()
                items_blog = data_blog['items']

                if not items_blog:  # 검색 결과가 더 이상 없으면 반복 중단
                    break

                for item in items_blog:
                    title = item['title'].replace('<b>', '').replace('</b>', '')
                    description = item['description'].replace('<b>', '').replace('</b>', '')
                    link = item['link']
                    writer.writerow([query, '블로그', title, description, link])

            else:
                print(f"블로그 에러 코드: {response_blog.status_code}")
                break  # 에러 발생 시 반복 중단

print(f"{filename} 파일 저장 완료.")

In [ ]:
df = pd.read_csv('뉴스_블로그_크롤링.csv')
print(df.head())
print("\n", df.shape)

# 불용어 처리 및 전처리

In [ ]:
# 불용어 목록 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '지역',
             '있', '하', '것', '그', '되', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '지', '대', '그렇', '위하',
             '직업', '진로', '최대', '그리고', '또한', '하지만', '그래서', '그러나', '때문에', '즉', '다시', '말', '하면', '혹은', '또', '더',
             '이런', '저런', '그런', '이', '그', '저', '것들', '뭐', '왜', '어떻게', '어떤', '여기', '저기', '거기', '누가', '무엇', '어디', '언제',
             '어느', '모든', '각', '그리고', '따라서', '그러므로', '이어서', '뿐만', '아니라', '하지만', '그러나', '이렇', '저렇', '그렇지만', '따라',
             '위해', '수', '있다', '없다', '이기', '때문', '때문에', '대한', '중장년', '은퇴', '퇴직자', '구직', '추천', '교육', '관련', '정보',
             '자료', '연구', '분석', '통계', '결과', '보고서', '사례', '설명', '비교', '평가', '방법', '방안', '내용', '항목', '기준', '목적',
             '요약', '개요', '소개', '특징', '특성', '현황', '문제', '해결', '방향', '목표', '전략', '계획', '발전', '개발', '이슈', '동향',
             '추세', '경향', '영향', '역할', '필요', '중요', '주요', '중심', '기본', '핵심', '중점', '주체', '대상', '대책', '정책', '방침',
             '상황', '조건', '환경', '기회', '위기', '제도', '관리', '운영', '시스템', '프로그램', '서비스', '지원', '제공', '협력', '기술', '참여',
             '활동', '비용', '경비', '예산', '효과', '효율', '성능', '성과', '결과', '평가', '진단', '분류', '구분', '분석', '사업', '시행',
             '대표', '대학', '대학', '자기계발', '취미', '취미생활', '관심사', '추진', '본격', '플랫폼', '문화', '마음', '에서', '협동조합',
             '평생교육', '커뮤니티', '어려움', '친구', '사회적', '고립', '통해', '&quot;']

# 수집된 데이터 파일 읽기
filename = "뉴스_블로그_크롤링.csv"
df = pd.read_csv(filename)

# 형태소 분석기 초기화
okt = Okt()

# 불용어 및 특수문자 제거 함수
def clean_text(text):
    if not isinstance(text, str):
        return ''  # 문자열이 아닌 경우 빈 문자열 반환

    tokens = okt.morphs(text) # 텍스트 토큰화

    tokens = [token for token in tokens if token not in stopwords] # 불용어 제거
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens] # 특수문자 제거
    tokens = [token for token in tokens if token] # 빈 문자열 제거

    return ' '.join(tokens) # 토큰을 다시 하나의 문자열로 결합

# 제목과 설명에서 불용어 및 특수문자 제거
df['제목'] = df['제목'].apply(clean_text)
df['설명'] = df['설명'].apply(clean_text)

# 새로운 파일에 저장
processed_filename = "크롤링_전처리.csv"
df.to_csv(processed_filename, index=False, encoding='utf-8-sig')

print(f"{processed_filename} 파일 저장 완료.")

크롤링_전처리.csv 파일 저장 완료.


In [ ]:
df = pd.read_csv('크롤링_전처리.csv')
print(df.head())
print("\n", df.shape)

         검색어  출처                                    제목  \
0  중장년 취미 생활  뉴스              성북구 중장 1 인 가구 힐링 원데이 클래스   
1  중장년 취미 생활  뉴스      성북구 중장 1 인 가구 셀프 집 수리 힐링 원데이 클래스   
2  중장년 취미 생활  뉴스                   성북구 중장 1 인 가구 맞춤 야간   
3  중장년 취미 생활  뉴스  quot 미래 일단 잡자 quot MZ 세대 겨냥 이색 보험 뜬다   
4  중장년 취미 생활  뉴스          전북 비정규직 인원 3만 명대 육박 프리터 족 급증   

                                                  설명  \
0  향후 성북구 고려대학교 KU 건강 연구소 해 우울 감 을 경험 하는 중장 집단 심리...   
1  성북구 향후 고려대학교 KU 건강 연구소 해 우울 감 을 경험 하는 중장 집단 심리...   
2  내년 에는 동아리 을 하는 1 인 가구 생활 뿐 만 건강 까지 챙긴다는 이다 이승 ...   
3  윤씨 자산 중장 년층 비해 부족하지만 하고 싶은 일 많은 20 대들을 위 보험 을 ...   
4  회사 다닐 보다 시간 구애 받지 않고 자유로운 여가 생활 을 즐기는 여유 많아져 관...   

                                                  링크  
0  https://n.news.naver.com/mnews/article/081/000...  
1  https://n.news.naver.com/mnews/article/421/000...  
2  https://n.news.naver.com/mnews/article/629/000...  
3  https://www.greened.kr/news/articleView.html?i...  
4  http://www.domin.co.kr/news/articleV